In [ ]:
import numpy as np

def simulated_annealing_augmented(
    Q_base, map_edge, map_z,
    lambda_deg, lambda_z, lambda_lam, lambda_E,
    rho, v, k, lam, mu, E_target,
    restarts=8, T0=20.0, cooling=0.9998, max_iters=200000
):

    import numpy as np

    N = Q_base.shape[0]
    best_x = None
    best_L = np.inf

    # Función auxiliar para evaluar L(x)
    def L_of(x):
        return energia_aumentada(
            x, Q_base,
            lambda_deg, lambda_z, lambda_lam, lambda_E,
            rho, map_edge, map_z,
            v, k, lam, mu, E_target
        )

    for r in range(restarts):
        # Estado inicial aleatorio
        x = np.random.randint(0, 2, size=N)
        L = L_of(x)
        T = T0

        # Bucle de Metropolis
        for _ in range(max_iters):
            p = np.random.randint(N)
            x_new = x.copy()
            x_new[p] = 1 - x[p]
            L_new = L_of(x_new)
            dL = L_new - L
            # criterio de aceptación
            if dL < 0 or np.random.rand() < np.exp(-dL / T):
                x, L = x_new, L_new
            T *= cooling
            if T < 1e-6:
                break

        # Guarda la mejor solución de todos los restarts
        print(f"[Aug-SA] Restart {r+1}: L = {L:.2f}")
        if L < best_L:
            best_L = L
            best_x = x.copy()

    print(f"[Aug-SA] Mejor L final = {best_L:.2f}")
    return best_x
#------------------------------
#Función auxiliar: build_A
#------------------------------
def build_A(x, map_edge, v):
    """
    Reconstruye la matriz de adyacencia A (v×v) a partir del vector x
    y del diccionario map_edge que mapea (i,j) → p.
    """
    A = np.zeros((v, v), dtype=int)
    for (i, j), p in map_edge.items():
        A[i, j] = A[j, i] = x[p]
    return A
def eval_grados(x, map_edge, v, k):
    g_deg = []
    A = build_A(x, map_edge, v)
    for i in range(v):
        g_deg.append(int(A[i].sum()) - k)
    return np.array(g_deg)

def eval_ancillas(x, map_edge, map_z):
    g_z = []
    for (i,j,m), pz in map_z.items():
        pim = map_edge[(min(i,m),max(i,m))]
        pjm = map_edge[(min(j,m),max(j,m))]
        #g_z.append(int(x[pim] + x[pjm] - 2*x[pz]))
        g_z.append(int(x[pim]*x[pjm] +x[pim]*x[pz]+x[pz]*x[pjm]- 3*x[pz]))
    return np.array(g_z)


##### CORREGIR ESTA PARTE
def eval_vecinos(x, map_edge, map_z, v, lam, mu):
    g_lam = []
    for i in range(v):
        for j in range(i+1,v):
            pij = map_edge[(i,j)]
            zs  = [map_z[(i,j,m)] for m in range(v) if m not in (i,j)]
            S   = np.sum(x[z] for z in zs)
            T   = lam*x[pij] + mu*(1-x[pij])
            g_lam.append(int(S - T))
    return np.array(g_lam)

def eval_total_aristas(x, E):
    return np.array([int(x.sum() - E)])

#CORREGIR ESTA PARTE
def energia_aumentada(x, Q_base,
                      lambda_deg, lambda_z, lambda_lam, lambda_E,
                      rho, map_edge, map_z,
                      v, k, lam, mu, E):
    # 1) término QUBO base
    E0 = x @ (Q_base @ x)
    # 2) recoger todas las g's
    g1 = eval_grados(x,map_edge,v,k)
    g2 = eval_ancillas(x,map_edge,map_z)
    g3 = eval_vecinos(x,map_edge,map_z,v,lam,mu)
    g4 = eval_total_aristas(x,E)
    # 3) construir L = E0 + sum λ g + (ρ/2) sum g^2
    L = E0
    for g,lam_v in zip([g1,g2,g3,g4],
                       [lambda_deg, lambda_z, lambda_lam, lambda_E]):
        L += lam_v.dot(g) + 0.5*rho*(g*g).sum()
    return L

#1) precálculo Q_base, map_edge, map_z
Q_base, map_edge, map_z = generar_qubo(v,k,lam,mu,
                                       B_deg,B_z,C)

Q_base = añadir_penal_global(Q_base, v,k,D)
E_target = vk//2

#2) inicializar λ y ρ
lambda_deg = np.zeros(v)
lambda_z   = np.zeros(len(map_z))
lambda_lam = np.zeros(v(v-1)//2)
lambda_E   = np.zeros(1)
rho        = 10.0

for it in range(10):   # 10 iteraciones de ALM
    # a) minimizar L en x (simulated annealing sobre energía_aumentada)
    x = simulated_annealing_augmented(
          Q_base, map_edge, map_z,
          lambda_deg, lambda_z, lambda_lam, lambda_E,
          rho, v, k, lam, mu, E_target,
          restarts=8, T0=20., cooling=0.9998, max_iters=200000
        )
    # b) evaluar violaciones
    g1 = eval_grados(x,map_edge,v,k)
    g2 = eval_ancillas(x,map_edge,map_z)
    g3 = eval_vecinos(x,map_edge,map_z,v,lam,mu)
    g4 = eval_total_aristas(x,E_target)
    # c) actualizar multiplicadores
    lambda_deg += rho * g1
    lambda_z   += rho * g2
    lambda_lam += rho * g3
    lambda_E   += rho * g4
    # d) opcional: aumentar ρ
    rho *= 2
    # e) chequea convergencia
    tot_viol = np.sum(np.abs(g1))+np.sum(np.abs(g2))+np.sum(np.abs(g3))+abs(g4[0])
    print(f"Iter {it}: violaciones totales = {tot_viol}")
    if tot_viol==0:
        print("¡Convergió a solución exacta!")
        break